In [1]:
!pip install llama-index --quiet
!pip install duckduckgo-search llama-index-readers-web --quiet
!pip install llama-index-embeddings-huggingface --quiet
!pip install llama-index-graph-stores-neo4j neo4j --quiet


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
from llama_index.readers.web import SimpleWebPageReader
from duckduckgo_search import DDGS

def get_urls_for_topic(topic, max_results=5):
    results = DDGS().text(topic, max_results=max_results)
    return [res['href'] for res in results]

topic = "shakespeare"
urls = get_urls_for_topic(topic)

documents = SimpleWebPageReader(html_to_text=True).load_data(urls)

In [3]:
len(documents)

5

In [15]:
from llama_index.core.indices.property_graph import ImplicitPathExtractor

kg_extractor = ImplicitPathExtractor()

In [8]:
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="intfloat/e5-base-v2")

splitter = SemanticSplitterNodeParser(
    buffer_size=1,
    breakpoint_percentile_threshold=95,
    embed_model=embed_model
)
nodes = splitter.get_nodes_from_documents(documents)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [9]:
len(nodes)

135

In [ ]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.core import StorageContext, PropertyGraphIndex

NEO4J_URI = "XXXX"
NEO4J_USERNAME = "XXXX"
NEO4J_PASSWORD = "XXXX"

graph_store = Neo4jPropertyGraphStore(
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    url=NEO4J_URI,
)

storage_context = StorageContext.from_defaults(property_graph_store=graph_store)

index = PropertyGraphIndex(
    nodes=nodes,
    kg_extractors=[kg_extractor],
    embed_model=embed_model,
    storage_context=storage_context,
    use_async=False,
)